In [20]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np 
import glob as glob
import random

In [7]:
def gsmooth(x_array, y_array, var_y, vexp = .001, nsig = 5.0):
    #returns smoothed flux array
    # if no variance array create a constant one
    if len(var_y) == 0:
        var_y = np.zeros(len(y_array))
        
    for i in range(len(var_y)):
        if var_y[i] == 0:
            var_y[i] = 1E-20
            # var_y[i] = 1E-31
    
    # Output y-array
    new_y = np.zeros(len(x_array), float)
    
    # Loop over y-array elements
    for i in range(len(x_array)):
        
        # Construct a Gaussian of sigma = vexp*x_array[i]
        gaussian = np.zeros(len(x_array), float)
        sigma = vexp*x_array[i]
        
        # Restrict range to +/- nsig sigma
        sigrange = np.nonzero(abs(x_array-x_array[i]) <= nsig*sigma)
        gaussian[sigrange] = (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-0.5*((x_array[sigrange]-x_array[i])/sigma)**2)
        
        # Multiply Gaussian by 1 / variance
        W_lambda = gaussian / var_y
        
        # Perform a weighted sum to give smoothed y value at x_array[i]
        W0 = np.sum(W_lambda)
        W1 = np.sum(W_lambda*y_array)
        new_y[i] = W1/W0

    # Return smoothed y-array
    return new_y

In [8]:
def deredshift(wavelength, z):
    dered_wave = wavelength / (1+z)
    return dered_wave

In [9]:
def find_min_wave(dered_wave, sm_flux, w1 = 5900., w2 = 6500.):
    elem_range = np.where((dered_wave > w1) & (dered_wave < w2)) #set domain as within minwave and maxwave
    list(elem_range)
    elem_wavelist = dered_wave[elem_range] 
    elem_fluxlist = sm_flux[elem_range] #find corresponding flux vales for wavelengths within domain
    elem_flux = min(elem_fluxlist) #find minimum value within these flux vales to locate "dip"
    elem_wavelength = dered_wave[np.where(sm_flux == elem_flux)][0] #find the corresponding wavelength
    return(elem_wavelength)

In [10]:
def find_velocity(min_wave, rest_wave = 6355.):
    wavelength_prop = min_wave / rest_wave
    ejecta_velocity = c * (wavelength_prop ** 2 - 1) / (wavelength_prop ** 2 + 1)
    return ejecta_velocity

In [43]:
def get_SN_name(file):
    SN_name = file.split('-')[0]
    return SN_name

In [39]:
def make_z_dict(file):
    #reads in CfA4_info.txt
    sn_datatext = open(file, "r")
    sn_datainfo = sn_datatext.readlines()
    start = False
    for line in sn_datainfo:
        if line.split()[0] == '2006ct':
                start = True
        if start:
            line_elems = line.split()
            name = line_elems[0]
            z = line_elems[10]
            z_dict[name] = float(z)
    return z_dict

In [32]:
def vel_error_dist(wavelength, flux, flux_error, initialvexp, n = 50):
    velocity_distribution = []
    minwave = 6000
    maxwave = 7000
    for i in range(n):
        vexp= random.uniform(0,initialvexp)
        smooth_flux = gsmooth(wavelength, flux, flux_error,vexp)
        SiII_obs = find_min_wave(wavelength, smooth_flux)
        v = find_velocity(SiII_obs)
        print (SiII_obs, vexp, v)
        velocity_distribution.append(v) #we can delete this line if we don't want to analyze the velocities individually
        plt.hist(velocity_distribution)
    return velocity_distribution

In [35]:
#TEST YOUR CODE ON ONE OBJECT (plot original spectrum, deredshifted spectrum, and the smoothed deredshifted spectrum, print velocity)
#YOUR CODE GOES HERE

In [40]:
#FINAL SCRIPT STEPS
#1) Get array of all flm file names
#2) Make redshift dictionary
#3) loop through files
    #a) read in spectrum
    #b) get SN name from file name
    #c) get redshift from dictionary
    #d) deredshift
    #e) smooth
    #f) find wavelength where Si line is minimum
    #g) find and print velocity (and SN name)

In [41]:
#YOUR CODE GOES HERE
z_dict = make_z_dict("CfA4_info.txt")
sndata = glob.glob("*.flm")
wavelength, flux, flux_error = np.loadtxt(sndata[0], unpack = True)
z = 0.017192 #obtain from NED
c = 299792 #speed of light in km/s
print(sndata)
dered_wave = deredshift(wavelength, z)
sm_flux = gsmooth(dered_wave, flux, flux_error)
min_wave = find_min_wave(dered_wave, sm_flux)
print(min_wave)
v = find_velocity(min_wave)
print(v)

NameError: name 'z_dict' is not defined

In [16]:
make_z_dict("CfA4_info.txt")

NameError: name 'z_dict' is not defined